# ConversationSummaryMemory
 
Hướng dẫn này bao gồm cách tóm tắt và quản lý lịch sử trò chuyện bằng cách sử dụng `LangChain`.

`ConversationSummaryMemory` tối ưu hóa việc sử dụng bộ nhớ bằng cách tóm tắt nội dung trò chuyện, cho phép quản lý hiệu quả lịch sử trò chuyện dài.

Trong hướng dẫn này, chúng ta sẽ trình bày cách triển khai chức năng tóm tắt trò chuyện bằng cách sử dụng `ConversationSummaryMemory` của LangChain.


In [1]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

load_dotenv(override=True, dotenv_path="../.env")

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.0)

## Conversation Summary Memory

Chắc chắn rồi, hãy cùng khám phá cách sử dụng một loại bộ nhớ phức tạp hơn: `ConversationSummaryMemory`.

Loại bộ nhớ này tạo ra **bản tóm tắt cuộc trò chuyện theo thời gian**, điều này có thể hữu ích để nén thông tin trò chuyện khi cuộc trò chuyện tiến triển.

`ConversationSummaryMemory` tóm tắt cuộc trò chuyện khi nó tiếp tục và **lưu trữ bản tóm tắt hiện tại trong bộ nhớ**.

Bộ nhớ này sau đó có thể được sử dụng để chèn lịch sử cuộc trò chuyện đã tóm tắt vào các prompt hoặc chains.

Nó đặc biệt hữu ích cho các cuộc trò chuyện dài hơn, nơi việc giữ lại toàn bộ lịch sử cuộc trò chuyện trong prompt sẽ chiếm quá nhiều tokens.

Hãy tạo một `ConversationSummaryMemory`.


In [2]:
from langchain.memory import ConversationSummaryMemory

memory = ConversationSummaryMemory(llm=llm, return_messages=True)

memory.save_context(
    inputs={"human": "What is the price of the Europe travel package?"},
    outputs={
        "ai": "The base price for the 14-night, 15-day Europe package is €3,500. This price includes airfare, hotel accommodations, and admission fees to designated tourist attractions. Additional costs may vary depending on optional tours or personal expenses."
    },
)
memory.save_context(
    inputs={"human": "What are the main tourist attractions included in the trip?"},
    outputs={
        "ai": "This trip includes visits to famous European landmarks such as the Eiffel Tower in Paris, the Colosseum in Rome, the Brandenburg Gate in Berlin, and Rhine Falls in Zurich. You'll comprehensively experience iconic sites in each city."
    },
)
memory.save_context(
    inputs={"human": "Is travel insurance included?"},
    outputs={
        "ai": "Yes, basic travel insurance is provided for all travelers. This insurance includes medical expense coverage and support in emergency situations. Enhanced coverage is available upon request."
    },
)
memory.save_context(
    inputs={
        "human": "Can I upgrade my flight seat to business class? How much does it cost?"
    },
    outputs={
        "ai": "Upgrading your flight seat to business class is possible. The upgrade cost is approximately €1,200 round-trip. Business class offers benefits such as wider seats, premium in-flight meals, and additional baggage allowance."
    },
)
memory.save_context(
    inputs={"human": "What is the hotel rating included in the package?"},
    outputs={
        "ai": "The package includes accommodation in 4-star hotels. Each hotel offers comfort and convenience, with central locations providing easy access to tourist sites. All hotels are equipped with excellent services and amenities."
    },
)
memory.save_context(
    inputs={"human": "Can you provide more details about the meal options?"},
    outputs={
        "ai": "This travel package includes daily breakfast served at the hotel. Lunch and dinner are not included, giving travelers the flexibility to explore and enjoy a variety of local cuisines. Additionally, a list of recommended restaurants in each city is provided to help maximize your culinary experience."
    },
)
memory.save_context(
    inputs={
        "human": "How much is the deposit for booking the package? What is the cancellation policy?"
    },
    outputs={
        "ai": "A deposit of €500 is required when booking the package. The cancellation policy allows a full refund if canceled at least 30 days before the booking date. After that, the deposit becomes non-refundable. If canceled within 14 days of the travel start date, 50% of the total cost will be charged, and after that, the full cost will be non-refundable."
    },
)

/tmp/ipykernel_786613/1394417146.py:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationSummaryMemory(llm=llm, return_messages=True)


In [3]:
# Check stored memory.
print(memory.load_memory_variables({})["history"])

[SystemMessage(content='The human asks about the price of the Europe travel package, which costs €3,500 for 14 nights and 15 days. Optional upgrades like business class seats are available for an additional €1,200 round-trip. The package includes accommodation in 4-star hotels, daily breakfast, and recommendations for local restaurants. A deposit of €500 is required to book the package, with a cancellation policy allowing a full refund if canceled at least 30 days before the travel date.', additional_kwargs={}, response_metadata={})]


## Conversation Summary Buffer Memory

`ConversationSummaryBufferMemory` kết hợp hai ý tưởng chính:

Nó giữ lại một bộ đệm của lịch sử trò chuyện gần đây trong bộ nhớ đồng thời biên dịch các tương tác cũ hơn thành một bản tóm tắt mà không xóa hoàn toàn chúng.

Nó xác định thời điểm xóa cuộc trò chuyện dựa trên **độ dài token**, thay vì số lượng tương tác.


In [4]:
from langchain.memory import ConversationSummaryBufferMemory

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=200,  # Set the token length threshold for summarization.
    return_messages=True,
)

/tmp/ipykernel_786613/4270176610.py:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationSummaryBufferMemory(


In [5]:
memory.save_context(
    inputs={"human": "What is the price of the Europe travel package?"},
    outputs={
        "ai": "The base price for the 14-night, 15-day Europe package is €3,500. This price includes airfare, hotel accommodations, and admission fees to designated tourist attractions. Additional costs may vary depending on optional tours or personal expenses."
    },
)

In [6]:
# Check the stored conversation history in memory
memory.load_memory_variables({})["history"]

[HumanMessage(content='What is the price of the Europe travel package?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='The base price for the 14-night, 15-day Europe package is €3,500. This price includes airfare, hotel accommodations, and admission fees to designated tourist attractions. Additional costs may vary depending on optional tours or personal expenses.', additional_kwargs={}, response_metadata={})]

In [7]:
memory.save_context(
    inputs={"human": "What are the main tourist attractions included in the trip?"},
    outputs={
        "ai": "This trip includes visits to famous European landmarks such as the Eiffel Tower in Paris, the Colosseum in Rome, the Brandenburg Gate in Berlin, and Rhine Falls in Zurich. You'll comprehensively experience iconic sites in each city."
    },
)
memory.save_context(
    inputs={"human": "Is travel insurance included?"},
    outputs={
        "ai": "Yes, basic travel insurance is provided for all travelers. This insurance includes medical expense coverage and support in emergency situations. Enhanced coverage is available upon request."
    },
)
memory.save_context(
    inputs={
        "human": "Can I upgrade my flight seat to business class? How much does it cost?"
    },
    outputs={
        "ai": "Upgrading your flight seat to business class is possible. The upgrade cost is approximately €1,200 round-trip. Business class offers benefits such as wider seats, premium in-flight meals, and additional baggage allowance."
    },
)
memory.save_context(
    inputs={"human": "What is the hotel rating included in the package?"},
    outputs={
        "ai": "The package includes accommodation in 4-star hotels. Each hotel offers comfort and convenience, with central locations providing easy access to tourist sites. All hotels are equipped with excellent services and amenities."
    },
)

In [8]:
# Check the stored conversation history in memory
memory.load_memory_variables({})["history"]

[SystemMessage(content="The human inquires about the price of the Europe travel package. The AI responds that the base price for the 14-night, 15-day Europe package is €3,500, which includes airfare, hotel accommodations, and admission fees to tourist attractions. Additional costs may vary depending on optional tours or personal expenses. The trip includes visits to famous European landmarks such as the Eiffel Tower in Paris, the Colosseum in Rome, the Brandenburg Gate in Berlin, and Rhine Falls in Zurich. You'll comprehensively experience iconic sites in each city.", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Is travel insurance included?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Yes, basic travel insurance is provided for all travelers. This insurance includes medical expense coverage and support in emergency situations. Enhanced coverage is available upon request.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='